# STDSR23 - Assignment-01

> Karina Denisova, BS21-DS-01, @karinadenisova

# Quantile implementation

February 2023

## Assignment description

* Obtain the research paper titled ["A Survey of Approximate Quantile Computation on Large-Scale Data"](https://arxiv.org/pdf/2004.08255.pdf)

* Select and implement one of the methods discussed in the paper, following good coding style and with appropriate comments. Your algorithm can be based on `BasicQuantileAlgorithm` class. In that case, your main implementation is in the `compute_quantile` function.
* Prepare a report that includes
    * A detailed explanation of the selected algorithm
    * A listing of the pros and cons of the chosen algorithm
* Test your implementation. More on that below in section `Test`.
* Submit Jupyter Notebook *.ipynb* to the Moodle

Note: Code style, comments, and overall organization of the report will be
taken into account in the grading process.


In [ ]:
from typing import List

class BasicQuantileAlgorithm:
    """
    Abstract class for quantile computing algorithm
    """
    
    def __init__(self) -> None:
        """
        Initializer for algorithm
        """
    
    def compute_quantile(self, q) -> float:
        """
        Implementation of some quantile algorithm
        """
        raise NotImplementedError('compute_quantile is not implemented')
    
    def compute(self, q) -> float:
        assert 0.0 <= q <= 1.0, f"q should be in [0;1]. Got {q}"
        """
        Compute the q-th quantile
        
        Hides implementation of _compute_quantile. 
        
        :param q: Quantile to compute, which must be between 0 and 1 inclusive
        :return: q-th quantile
        """
        return self.compute_quantile(q)
    
    # NOTE: multi_compute is just a basic function
    # for computing multiple quantile with one function call. 
    # This can be modified, so that multiple quantiles
    # calculation become efficient. 
    def multi_compute(self, qs) -> List[float]:
        """
        Compute multiple q-th quantiles
        
        :param qs: list of quantiles to compute, 
            all should be between 0 and 1 inclusive
        :return: list of computed quantiles
        """
        return [self.compute(q) for q in qs]

In [ ]:
import numpy as np

class NumpyQuantileAlgorithm(BasicQuantileAlgorithm):
    """
    Example of quantile algorithm
    
    This algorithm implementation is 
    based on the numpy.quantile. 
    """
    def __init__(self):
        """
        Initializer of the class. 
        
        It initializes self.data with empty list
        """
        self.data = []
    
    def add_item(self, item):
        """
        Adds an item to the existing data
        """
        self.data.append(item)
    
    def add_multiple_items(self, items):
        """
        Adds multiple items to the existing data
        """
        self.data += items
    
    def compute_quantile(self, q) -> float:
        """
        Compute the q-th quantile
        
        :param q: Quantile to compute, which must be between 0 and 1 inclusive
        :return: q-th quantile
        """
        return np.quantile(self.data, q)
    
    # NOTE: Here, for multi_compute we
    # can just call the same function
    # compute_quantile, because it could
    # take a vector of multiple quantiles.
    # This is more efficient, than basic 
    # function call, because of the 
    # vectorized implementation.
    def multi_compute(self, qs) -> List[float]:
        """
        Compute multiple q-th quantiles
        
        :param qs: list of quantiles to compute, 
            all should be between 0 and 1 inclusive
        :return: list of computed quantiles
        """
        return list(self.compute_quantile(qs))

In [ ]:
import math
import numpy as np


"""Custom quantile tuple class for representing ranges and quantities of values"""
class Tuple:
    """
    Initialize a new tuple
    """
    def __init__(self, value, gap, delta):
        self.value = value
        self.gap = gap
        self.delta = delta

    def __str__(self):
       return f"Tuple(value={self.value}, gap={self.gap}, delta={self.delta})"

    def __repr__(self):
        return self.__str__()

"""Greenwlad and Khanna's algorithm (GK01) for streaming data"""
class GK:
    def __init__(self, epsilon):
        self.epsilon = epsilon # The error factor. should be [0,1)
        self.tuples = [] # The list of tuples
        self.n = 0 #The number of observations 

    def estimator(self):
        return math.floor(1.0 / (2.0 * self.epsilon))

    def insert(self, v):
        """
        Locates the correct position in the summary data set for the observation v, and inserts a new tuple (v,1,floor(2en)) If v is the new minimum or maximum, then instead insert tuple (v,1,0).
        """
        t = Tuple(v, 1, math.floor(2.0 * self.epsilon * self.n))
        i = self.get_insert_index(v)
        if i == 0 or i == len(self.tuples):
            t.delta = 0
        self.tuples.insert(i, t)
        self.n += 1
        if self.should_compress():
            self.compress()

    def get_insert_index(self, v):
        """
        Returns the index of the tuple t in the list of tuples.
        Locates the proper position of v in a vector such that when v is inserted at position i, it is less then the element at i+1 if any, and greater than or equal to the element at i-1 if any.
        """
        if len(self.tuples) == 0:
            return 0
        if v <= self.tuples[0].value:
            return 0
        if v >= self.tuples[-1].value:
            return len(self.tuples)
        left = 0
        right = len(self.tuples) - 1
        while left < right:
            middle = math.floor((left + right) / 2)
            if v < self.tuples[middle].value:
                right = middle
            else:
                left = middle + 1
        return left

    def should_compress(self):
        """
        Returns True if the list of tuples should be compressed, False otherwise.
        """
        period = self.estimator()  # math.floor(1.0 / (2.0 * self.epsilon))
        return self.n % period == 0

    def compress(self):
        """
        Remove the redundant information about quantiles
        """
        s = len(self.tuples)
        for j in range(s - 2, 1):
            while j < len(self.tuples) - 1 and self.can_delete(j):
                pass

    def can_delete(self, i):
        """
        Check if the tuple at index i can be deleted. Remove the ith tuple from the self.tuple. Only permitted if g[i] + g[i+1] + delta[i+1] < 2 * epsilon * n
        """
        t1 = self.tuples[i]
        t_next = self.tuples[i + 1]
        threshold = math.floor(2.0 * self.epsilon * self.n)
        if t1.delta >= t_next.delta and t1.gap + t_next.gap + t_next.delta < threshold:
            self.tuples.pop(i)
            t_next.gap += t1.gap
            return True
        return False

    def compute_quantile(self, phi):
        """
        Computes the quantile of the data set.
        """
        if self.n == 0:
            return 0
        r = math.floor(phi * self.n)
        en = self.epsilon * self.n

        first = self.tuples[0]
        prev = first.value
        prev_rmin = first.gap

        for j in range(1, len(self.tuples)):
            t = self.tuples[j]
            rmax = prev_rmin + t.gap + t.delta
            if rmax > r + en:
                return prev
            prev_rmin += t.gap
            prev = t.value 
        return prev



> MY REPORT ABOUT THE ALGORITHM

My data structure supports two operators:
   1. insert(v): Add new element v
   2. compute_quantile(p): get the estimation of the pth quantile
insert operation simply add new tuple in the proper plase. And Since{vi} sorted we could find position of new element using binary search(it reqires O(log(n)) operations).

Also I use Compress operation  in my structure to maintain a low number of tuples.

If the number of currently observed elements is n, the max error of compute_quantile(p) is En(where E is parameter that defines percision of quantile estimator). This means that compute_quantile returns an element that belongs to the interval between elements with ranks (p - E)n and (p + E)n in the sorted list of observed elements.


Pros of choosing GK01 algo: 
1. Low memory requirement. 
2. Fast computation time. 
3. Accurate estimation of quantiles. 
4. Easy to implement.

Cons of choosing GK01 algo: 
1. Not suitable for datasets with high variability. 
2. Can be computationally expensive for large datasets. 
3. Can produce inaccurate estimates if the sample size is too small.
   

> Add comparison of the time and space consumption (with numpy algorithm). Plot them depending on the sample size or time if your algorithm is for time-series quantile calculation.

In [ ]:
import time
from sys import getsizeof
from pympler import asizeof

size = [100, 1000, 10000] # you can change the size of the array to the bigger ones, but is significatnly will increase time of executung. because for 1M, it executs 2 minutes.
time_gk_arr = []
time_np_arr = []
time_gk_e2_arr = []

space_gk_arr = []
space_np_arr = []
space_gk_e2_arr = []

for n in size:
    gk = GK(0.001) # creating a GK object with error 0.001
    gk_e2 = GK(0.2) # creating a GK object with error 0.2
    nump = NumpyQuantileAlgorithm() # creating a numpuy.quantile object for treking memory usage
    arr = np.random.normal(0, 1, n) # filling array with random, normaly distributed numbers and inserting them into the algos
    for i in range(0, n):
        gk.insert(arr[i])
        nump.add_item(arr[i])
        gk_e2.insert(arr[i])
        
    # compute execution times and memory usage for all the algos and put them into arrays    
    s_t = time.time()
    gk.compute_quantile(0.5)
    e_t = time.time()
    time_gk_arr.append(e_t - s_t)

    space_gk_arr.append(asizeof.asizeof(gk))

    s_t = time.time()
    gk_e2.compute_quantile(0.5)
    e_t = time.time()
    time_gk_e2_arr.append(e_t - s_t)

    space_gk_e2_arr.append(asizeof.asizeof(gk_e2))
    
    s_t = time.time()
    nump.compute_quantile(0.5)
    e_t = time.time()
    time_np_arr.append(e_t - s_t)

    space_np_arr.append(asizeof.asizeof(nump))

# print(time_gk_arr)
# print(time_gk_e2_arr)
# print(time_np_arr)
# print(space_gk_arr)
# print(space_gk_e2_arr)
# print(space_np_arr)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))


plt.subplot(131)
plt.plot(time_gk_arr, space_gk_arr, '-bo', color='m', label='Greenwlad-Khanna(e=0.001)')
plt.plot(time_gk_e2_arr, space_gk_e2_arr, '-bo', color='g', label='Greenwlad-Khanna(e=0.2)')
plt.plot(time_np_arr, space_np_arr, '-bo', color='pink', label='numpy')
plt.xlabel('Time (s)')
plt.ylabel('Space (bytes)')
plt.legend(shadow=True, fancybox=True)


plt.subplot(132)
plt.plot(size, space_gk_arr, '-bo', color='m')
plt.plot(size, space_np_arr, '-bo', color='pink')
plt.plot(size, space_gk_e2_arr, '-bo', color='g')

plt.xlabel('Size of array')
plt.ylabel('Space (bytes)')

plt.subplot(133)
plt.plot(size, time_gk_arr, '-bo', color='m')
plt.plot(size, time_np_arr, '-bo', color='pink')
plt.plot(size, time_gk_e2_arr, '-bo', color='g')

plt.xlabel('Size of array')
plt.ylabel('Time (s)')



plt.show()

## Test

Here you need to show that your algorithm is working properly.
1. Take `normal` distribution. Show, that with increase in sample size, quantiles calculated with your algorithms are become very close to the `inverse cdf` of the distribution. 

2. Perform the same experiment with any other continuous distribution (exponential, logistic, etc.)

In [ ]:
#comparison of inverse CDF for normal distribution
from scipy import stats
from statistics import NormalDist
# res = NormalDist(mu=1, sigma=0.5).inv_cdf(0.5)
# print(res)

size = [100, 1000, 10000, 100000] # testing sample sizes
quantiles_arr = [] #array with values of the quantiles
inverse_cdf_arr = [] #array with vquantiles of the inverse cdf

for n in size:
    gk = GK(0.001)  #creating a GK object with error 0.001
    arr = np.random.normal(0, 1, n) # filling array with random, normaly distributed numbers and inserting them into the algos
    for i in range(0, n):
        gk.insert(arr[i])
    inverse_cdf_arr.append(NormalDist.from_samples(arr).inv_cdf(0.5)) # calculating the inverse cdf of the array
    quantiles_arr.append(gk.compute_quantile(0.5)) # calculating the quantiles of the array

plt.plot(size, quantiles_arr, '-bo', color='m', label="GK01")
plt.plot(size, inverse_cdf_arr, '-bo', color='pink', label="inverse cdf")
plt.legend(shadow=True, fancybox=True)
plt.xlabel('sample size')
plt.ylabel('quantile')
plt.show()

In [ ]:
#comparison of inverse CDF for normal distribution(2nd, better way)
from scipy import stats

size = [100, 1000, 10000, 100000] # testing sample sizes
quantiles_arr = [] #array with values of the quantiles
inverse_cdf_arr = [] #array with vquantiles of the inverse cdf

for n in size:
    gk = GK(0.001)  #creating a GK object with error 0.001
    arr = np.random.normal(0, 1, n) # filling array with random, normaly distributed numbers and inserting them into the gk
    for i in range(0, n):
        gk.insert(arr[i])

    inverse_cdf_arr.append(stats.norm.ppf(0.5)) # calculating the inverse cdf of the array
    quantiles_arr.append(gk.compute_quantile(0.5)) # calculating the quantiles of the array


plt.plot(size, quantiles_arr, '-bo', color='m', label="GK01")
plt.plot(size, inverse_cdf_arr, '-bo', color='pink', label="inverse cdf")
plt.legend(shadow=True, fancybox=True)
plt.xlabel('sample size')
plt.ylabel('quantile')
plt.show()

In [ ]:
#comparison of inverse CDF for exponential distribution

from scipy import stats

size = [100, 1000, 10000, 100000, 1000000] # testing sample sizes
quantiles_arr = [] #array with values of the quantiles
inverse_cdf_arr = [] #array with vquantiles of the inverse cdf

for n in size:
    gk = GK(0.001)  #creating a GK object with error 0.001
    arr = np.random.exponential(1, n) # filling array with random, exponentialy distributed numbers and inserting them into the gk
    for i in range(0, n):
        gk.insert(arr[i])

    inverse_cdf_arr.append(stats.expon.ppf(0.5)) # calculating the inverse cdf of the array
    quantiles_arr.append(gk.compute_quantile(0.5)) # calculating the quantiles of the array


plt.plot(size, quantiles_arr, '-bo', color='m', label="GK01")
plt.plot(size, inverse_cdf_arr, '-bo', color='pink', label="inverse cdf")
plt.legend(shadow=True, fancybox=True)
plt.xlabel('sample size')
plt.ylabel('quantile')
plt.show()

Perform an experiment that shows the main purpose of the algorithm. 

For example, some algorithms are created especially for time series data, so you should show that your algorithm is working in time series environment properly.

In [ ]:
"""
as this was shown in the graphics earlier? for the small samples GK01, takes less time than numpy, => this is useful for such caces
"""

Perform an experiment that shows the main purpose of the algorithm. 

For example, some algorithms are created especially for time series data, so you should show that your algorithm is working in time series environment properly.

In [ ]:
"""
as this was shown in the graphics earlier? for the small samples GK01, takes less time than numpy, => this is useful for such caces
"""

- [ ] Submit to Moodle!